## Introduction

Given a set of agents and a set of tasks and for each pair (agent, task) a costs or profit value. Moreover each agent has a budget, whcih means that the sum of costs assigned to an agent can not exceed his budget.

The goal is to find an assigment of agents to tasks such that the overall profit is maximized. (More precisely each task is assigned to only one agent.)

Its known that the generalized assignment problem is NP-hard.

### Special cases

1. If all agent budgets and tasks costs are equal to one, then the problem is the [assignment problem](https://en.wikipedia.org/wiki/Assignment_problem)
1. If consts and profits of all tasks are equal for each agent, then the problem reduces to the [multiple knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem).
1. If there is only a single agent, then the problem reduces to the [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem)

Further introductory reading: https://en.wikipedia.org/wiki/Generalized_assignment_problem

## Abstract mathematical formulation (as integer program)

$$
\begin{array}{llr}
\min & \sum_{i,j} c_{ij} x_{ij} & \\
s.t. & \sum_j w_{ij} x_{ij} \leq t_i & \forall i\\
     & \sum_i x_{ij} = 1 & \forall j
\end{array}
$$

where we denoted by

- $I$ agent index set
- $J$ task index set

### Decision variables

- $x_{ij}$, binary and equal to one iff agent $i$ is assigned to tassk $j$

### Parameter

- $c_{ij}$ cost/profit of assigning agent $i$ to task $j$
- $w_{ij}$ assigned capacity/weight if agent $i$ is assigned to task $j$
- $t_i$ capacity of agent $i$


The first constraint $\sum_j w_{ij} x_{ij} \leq t_i$ represents the fact, that the assigned task to an agent do not exceed the agent capacity.
The second constraint $\sum_i x_{ij} = 1$ assures that each task gets assign.

Of course the objective $\sum_{i,j} c_{ij} x_{ij}$ represents the cost of the assignment and an optimal solution will be one with smallest possible costs.

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

In [ ]:
# global parameter
_number_of_agents = 4
_number_of_tasks = 6

# set seed for reproducibility
np.random.seed(101101)

In [ ]:
# functions data generation
def gen_agents(number_of_agents = _number_of_agents):
    """returns list of unique strings representing agents"""
    return ["a" + str(i) for i in range(1,number_of_agents+1)]

def gen_tasks(number_of_tasks = _number_of_tasks):
    """returns list of unique strings representing tasks"""
    return ["t" + str(i) for i in range(1,number_of_tasks+1)]

def gen_agent_capacity(number_of_agents = _number_of_agents, up_limit = 10):
    """
    returns array with randomly generated integers between 0 and param up_limit 
    representing agents capacities
    """
    return np.random.randint(up_limit+1, size=number_of_agents)

def gen_assignment_cost(number_of_agents = _number_of_agents, 
                        number_of_task = _number_of_tasks,
                        up_limit = 10):
    """
    returns array with randomly generated integers between 1 and param up_limit
    representing costs assigning agent  i to task j
    rows = agents, columns = tasks
    """
    return np.random.randint(low=1, high = up_limit, size = (number_of_agents, number_of_task))

def gen_assignment_capacity(number_of_agents = _number_of_agents, 
                        number_of_task = _number_of_tasks,
                        up_limit = 10):
    """
    returns array with randomly generated integers between 1 and param up_limit
    representing used capacity of agent i when assigned to task j
    rows = agents, columns = tasks
    REMARK: keep in mind the obvious observation: The problem gets infeasible, if there the 'assignment capacity' for a tasks is bigger than the agent capacity
    """
    return np.random.randint(low=1, high = up_limit, size = (number_of_agents, number_of_task))


def gen_data(number_of_agents, number_of_tasks, up_limit, solver):
    """
    returns dicionary holding all problem data and optimization parameter
    arrays converted to pandas Data frames 
    """
    agents = gen_agents(number_of_agents)
    tasks = gen_tasks(number_of_tasks)
    agent_capacity = gen_agent_capacity(number_of_agents, up_limit)
    assignment_cost = gen_assignment_cost(number_of_agents, number_of_tasks, up_limit)
    assignment_capacity = gen_assignment_capacity(number_of_agents, number_of_tasks, up_limit)
    
    data = {
        'agents': agents,
        'tasks': tasks,
        'agent_capacity': pd.DataFrame(data = agent_capacity, index = agents, columns = ['agent_capacity']),
        'assignment_costs': pd.DataFrame(data = assignment_cost, index = agents, columns = tasks),
        'assignment_capacity': pd.DataFrame(data = assignment_capacity, index = agents, columns = tasks),
        'model_name': 'gerneralized assignment problem',
        'solver': solver
                                         
    }
    return data


In [ ]:
# I/O functions
def extract_solution(m):
    """
    extracts assignment and objective value
    """
    assignment = [(i,j) for i in m.I for j in m.J if pyo.value(m.x[i,j]) != 0]
    obj_val = pyo.value(m.OBJ)
    output = {
        'assignment': assignment,
        'objective_value': obj_val
    }
    return output

## Implementation of the generalized assignment problem in pyomo

In [ ]:
# pyomo implementation of a generalized assignment problem
def gap(data):
    #instanciate model
    m = pyo.ConcreteModel(data['model_name'])
    
    # define sets
    m.I = pyo.Set(initialize = data['agents'], doc = 'agents')
    m.J = pyo.Set(initialize = data['tasks'], doc = 'tasks')
    
    # decision variables
    m.x = pyo.Var(m.I, m.J, domain = pyo.Binary, doc = '1 iff agent i is assigned to target j')
    
    # define parameter
    @m.Param(m.I, doc = 'agent capacity')
    def t(m,i):
        return data['agent_capacity'].loc[i,'agent_capacity']
    @m.Param(m.I,m.J, doc = 'assignment costs')
    def c(m,i,j):
        return data['assignment_costs'].loc[i,j]
    @m.Param(m.I, m.J, doc = 'assignment capactity for agent i to task j')
    def w(m,i,j):
        return data['assignment_capacity'].loc[i,j]
    
    # define constraints
    @m.Constraint(m.I, doc = 'individual agent capacity constraint')
    def c1(m,i):
        return m.t[i] >= pyo.quicksum(m.x[i,j] * m.w[i,j] for j in m.J)
    @m.Constraint(m.J, doc = 'each tasks gets assigned')
    def c2(m,j):
        return  1 == pyo.quicksum(m.x[i,j] for i in m.I)
    
    # define objective
    m.OBJ = pyo.Objective(expr = sum(m.x[i,j] * m.c[i,j] for i in m.I for j in m.J), 
                          sense = pyo.minimize)
    
    # choose and apply solver
    solver = pyo.SolverFactory(data['solver'])
    solver.solve(m)
    
    return m

# Example and simple infeasiblity checks

For the simplicity of this example we randomly generate data.

Obviously the problem become infeasible if there exists a tasks $j$ no agent can perform, i.e. its required capacity is bigger than the agent capacities.

In more mathematical terms the problem is infeasible if there is a $j$ and such that for all $i$
$$
t_i \leq w_{ij}
$$

Because the data is randomly generated, we check the data for this type of infeasiblity too.

A short word for readers not trained in mathematics: This test is a necessary condition for infeasibility. If its passed, its means this kind of feasibility does not exists, but it does not mean that the problem is feasiblie. Another example of data for an infeasible problem is given, if the sum of the agent capacity is smaller than "the sum of the required capacity". 

To make it more explicit: Suppose all agents capacity is 2 and all tasks require a capacity of two. In this case the above test is passed, but if additionally there are more tasks than  agents there obviously does not exists an assignment of agents to tasks.

The function for our data test is given in the next code cell.

In [ ]:
# functions testing
## include one test for completeness checking if the problem data defines an infeasible problem

def test_data_1(data):
    """
    tests a necessary condition for infeasibility, passing test does not guarantee feasibility
    test if for each tasks there is at least one agent with sufficient high capacity 
    """
    # extract data
    assign_costs = data['assignment_costs']
    capactity = data['agent_capacity']
    # compute test
    ## vectorwise comparison, recall if difference is greater zero iff agent capacity is bigger than assign cap to task (row = agents)
    test = assign_costs.apply(lambda x: x - capactity['agent_capacity'], axis = 0) 
    test = test.apply(lambda x: x > 0, axis = 0) 
    ## convert to dictionary in order to return problematic columns/tasks (i.e. easier debugging)
    test = {x: any(test[x]) for x in test.columns}
    
    #define assertion error message
    exception = ('infeasible problem data, tasks with to high assignment costs and no agent with sufficiently high capacity exists', 
                 ' problem in ', [col for col in test.keys() if not test[col]])
    assert all(test), exception
    return all(test)

In [ ]:
# generate and test data
data = gen_data(_number_of_agents,_number_of_tasks, 10, 'cbc')
print('Data generarted and test passed:', test_data_1(data))

Data generarted and test passed: True


Lets have a look at the generated data.
But before we introduce our notation: The suffix a is used for agents and the suffix t is used for targets.

In [ ]:
print('Agent capacities:')
data['agent_capacity']

Agent capacities:


,agent_capacity
a1,10
a2,1
a3,10
a4,6


In [ ]:
print('Used capacity when agent i  is assigned to task j:')
data['assignment_capacity']

Used capacity when agent i  is assigned to task j:


,t1,t2,t3,t4,t5,t6
a1,7,4,5,9,5,4
a2,4,1,7,3,8,5
a3,8,5,2,3,2,8
a4,7,9,2,2,7,8


In [ ]:
print('Assignment costs:')
data['assignment_costs']

Assignment costs:


,t1,t2,t3,t4,t5,t6
a1,7,8,6,4,1,5
a2,7,1,2,7,2,1
a3,7,8,2,4,3,8
a4,7,1,7,7,7,4


Lets have a look at the optimal assignment for this problem data and have a look at the solution for different solvers.

In [ ]:
%%time
m = gap(data)

CPU times: user 10.4 ms, sys: 20.6 ms, total: 31 ms
Wall time: 111 ms


In [ ]:
extract_solution(m)

{'assignment': [('a1', 't5'),
  ('a1', 't6'),
  ('a2', 't2'),
  ('a3', 't1'),
  ('a3', 't3'),
  ('a4', 't4')],
 'objective_value': 23.0}

In [ ]:
data['solver'] = 'scip'

In [ ]:
%%time
m = gap(data)

CPU times: user 13.6 ms, sys: 19.6 ms, total: 33.2 ms
Wall time: 91.4 ms


In [ ]:
extract_solution(m)

{'assignment': [('a1', 't5'),
  ('a1', 't6'),
  ('a2', 't2'),
  ('a3', 't1'),
  ('a3', 't3'),
  ('a4', 't4')],
 'objective_value': 23.0}

## Remark on solvers and problem sizes.

- section status: draft:

- tested 3 oos solver: glpk, cbc and scip
- tested on different problem sizes (up to 500 agents, 800 tasks) and different random input data
- cbc and scip doing "well", small instances in ms, above bigger size ~30 sec
- glpk only applicable for small instances, but here as quick as the other


# next steps

- compare LP with apprioxmative solutions (does matching include a suitable one?)
- compare with alternative formulations, e.g.
    - https://www.diva-portal.org/smash/get/diva2:21391/FULLTEXT01.pdf, enthält refomrulierung as set partioon problem
- (could:) show solver solution times for different instances
    - including a paragraph on the complexity